### Installation
The only Python package you need for now is pydantic_ai.

In [2]:
!pip install pydantic_ai -qU

### Set Environment Ready
To make the demo application run ,we will also need `net-asyncio`.
Next setp is to set up environment variable `GEMINI_API_KEY` so that the Pydantic AI agents can pick it up in using gemini models.

In [3]:
!pip install nest-asyncio -qU

In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import os

# [Optinal] set proxy
proxy = os.getenv("PROXY_VALUE")

for proxy_var in ['HTTP_PROXY', 'HTTPS_PROXY', 'http_proxy', 'https_proxy']:
    os.environ[proxy_var] = proxy


### Pydantic AI Agents
Let's start looking into some cool examples of Pydantic AI agents.

### The Simplest One
Chat with Gemini `gemini-1.5-flash` straight away.

In [5]:
from pydantic_ai import Agent
from pydantic_ai.models.gemini import GeminiModel
# set gemini api key 
gemini_api_key = os.getenv("GEMINI_API_KEY")
os.environ["GEMINI_API_KEY"] = gemini_api_key
model = GeminiModel("gemini-1.5-flash")
agent = Agent(model)

In [7]:
response = agent.run_sync("Hey, There! I from chengdu!")
print(response.data)

Hello!  It's great to hear from someone in Chengdu!  你好！ (Nǐ hǎo!)



### Agent with Static Prompt

In [8]:
agent = Agent("gemini-1.5-flash", system_prompt="You can only speak Chinese")
response = agent.run_sync("Hey, dude!")
print(response.data)

嘿，哥们儿！



### Agent with Dynamic prompt

In [9]:
from pydantic_ai import Agent, RunContext

In [10]:
dynamic_prompt_agent = Agent("gemini-1.5-flash")

@dynamic_prompt_agent.system_prompt
def set_agent_name(ctx: RunContext[str]) -> str:
    return f"You name is {ctx.deps}."

response = dynamic_prompt_agent.run_sync("Hey, dude! Who are you?", deps="Jarvis")
print(response.data)

Hey there!  I'm Jarvis, an AI assistant.  How can I help you today?



### Agent with Dependency Type

In [11]:
from dataclasses import dataclass

@dataclass
class Player:
    name: str
    goals: int
    
agent = Agent(
    'gemini-1.5-flash',
    deps_type=Player,
    result_type=bool,
)

@agent.system_prompt
def add_player_name(ctx: RunContext[Player]) -> str:
    player_name = ctx.deps.name
    return f"The player's name is {player_name}."

@agent.system_prompt
def add_player_goals(ctx: RunContext[Player]) -> str:
    goals = ctx.deps.goals
    return f"The player's golas sofar is {goals}."

response = agent.run_sync("Hey, dude! Does the player ever score a goal?", deps=Player(name="Messi", goals=2))
print(response.data)

response = agent.run_sync("Hey, dude! Does the player ever score a goal?", deps=Player(name="Ronaldo", goals=0))
print(response.data)


True
False


### Agent result specified data structure

In [18]:
from pydantic import BaseModel, Field
from pydantic_ai import Agent

    
# Expect response model   
class FlowerDescription(BaseModel):
    flower_type: str = Field(description="flower type")
    price: int = Field(description="flower price")
    description: str = Field(description="description")
    reason: str = Field(description="Why recommend")
    

agent = Agent(
    'gemini-1.5-flash',
    result_type=FlowerDescription,
)

flower = "茉莉花"
price = "99.88"

result = agent.run_sync(f"你是一位专业的鲜花文案写撰写员，你能根据花名为{flower}，售价为{price}的花提供一个吸引人的简短中文描述吗？")
print(result.data.model_dump())
    


{'flower_type': '茉莉花', 'price': 9988, 'description': '清雅茉莉，幽香袭人，99.88元，带给你夏日清凉与浪漫。', 'reason': '茉莉花以其独特的香气和优雅的外形，深受大众喜爱，是送礼佳品。'}
